# **Prerequisites**

**Remembert that we have to always include step one, two, and three of last week's lecture. Why? Because we are using the free version of Google Colab.**

In [1]:
#Step 1: Install Dependencies
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.3.0/spark-3.3.0-bin-hadoop3.tgz
!tar xf spark-3.3.0-bin-hadoop3.tgz
!pip install -q findspark

#Step 2: Add environment variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "spark-3.3.0-bin-hadoop3"

#Step 3: Initialize Pyspark
import findspark
findspark.init()

In [2]:
#creating spark context
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
sc = spark.sparkContext
sc

<SparkContext master=local[*] appName=pyspark-shell>

# **Spark MLlib - Linear Regression (Part 2)**

# Data Transformations

You won't always get data in a convienent format, often you will have to deal with data that is non-numerical, such as customer names, or zipcodes, country names, etc...

A big part of working with data is using your own domain knowledge to build an intuition of how to deal with the data, sometimes the best course of action is to drop the data, other times feature-engineering is a good way to go, or you could try to transform the data into something the Machine Learning Algorithms will understand.

Spark has several built in methods of dealing with thse transformations, check them all out here: http://spark.apache.org/docs/latest/ml-features.html

Let's see some examples of all of this!

In [3]:
from pyspark.sql import SparkSession

In [4]:
spark = SparkSession.builder.appName('data').getOrCreate()

In [6]:
df = spark.read.csv('fake_customers.csv',inferSchema=True,header=True)

In [ ]:
df.show()

## **Data Features**

### StringIndexer

We often have to convert string information into numerical information as a categorical feature. This is easily done with the StringIndexer Method:

In [8]:
from pyspark.ml.feature import StringIndexer

df = spark.createDataFrame(
    [(0, "a"), (1, "b"), (2, "c"), (3, "a"), (4, "a"), (5, "c")],
    ["user_id", "category"])

indexer = StringIndexer(inputCol="category", outputCol="categoryIndex")
indexed = indexer.fit(df).transform(df)
indexed.show()

+-------+--------+-------------+
|user_id|category|categoryIndex|
+-------+--------+-------------+
|      0|       a|          0.0|
|      1|       b|          2.0|
|      2|       c|          1.0|
|      3|       a|          0.0|
|      4|       a|          0.0|
|      5|       c|          1.0|
+-------+--------+-------------+




**The next step would be to encode these categories into "dummy" variables.**

### VectorIndexer

VectorAssembler is a transformer that combines a given list of columns into a single vector column. It is useful for combining raw features and features generated by different feature transformers into a single feature vector, in order to train ML models like logistic regression and decision trees. VectorAssembler accepts the following input column types: all numeric types, boolean type, and vector type. In each row, the values of the input columns will be concatenated into a vector in the specified order. 

Assume that we have a DataFrame with the columns id, hour, mobile, userFeatures, and clicked:

     id | hour | mobile | userFeatures     | clicked
    ----|------|--------|------------------|---------
     0  | 18   | 1.0    | [0.0, 10.0, 0.5] | 1.0
     
userFeatures is a vector column that contains three user features. We want to combine hour, mobile, and userFeatures into a single feature vector called features and use it to predict clicked or not. If we set VectorAssembler’s input columns to hour, mobile, and userFeatures and output column to features, after transformation we should get the following DataFrame:

     id | hour | mobile | userFeatures     | clicked | features
    ----|------|--------|------------------|---------|-----------------------------
     0  | 18   | 1.0    | [0.0, 10.0, 0.5] | 1.0     | [18.0, 1.0, 0.0, 10.0, 0.5]

In [ ]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

dataset = spark.createDataFrame(
    [(0, 18, 1.0, Vectors.dense([0.0, 10.0, 0.5]), 1.0)],
    ["id", "hour", "mobile", "userFeatures", "clicked"])
dataset.show()

In [11]:
assembler = VectorAssembler(
    inputCols=["hour", "mobile", "userFeatures"],
    outputCol="features")

output = assembler.transform(dataset)
print("Assembled columns 'hour', 'mobile', 'userFeatures' to vector column 'features'")
output.select("features", "clicked").show()

Assembled columns 'hour', 'mobile', 'userFeatures' to vector column 'features'
+--------------------+-------+
|            features|clicked|
+--------------------+-------+
|[18.0,1.0,0.0,10....|    1.0|
+--------------------+-------+



There are many more data transformations available, we will cover them once we encounter a need for them, for now these were the most important ones.

Let's continue on to Linear Regression!

## **Linear Regression Example**

Let's walk through the example below. Doing this will help you understand it, and then apply it to your own problems.

In [12]:
from pyspark.sql import SparkSession

In [13]:
spark = SparkSession.builder.appName('lr_example').getOrCreate()

In [14]:
from pyspark.ml.regression import LinearRegression

In [15]:
# Load training data
training = spark.read.format("libsvm").load("sample_linear_regression_data.txt")

Interesting! We haven't seen libsvm formats before. In fact they aren't very popular when working with datasets in Python, but the Spark Documentation makes use of them a lot because of their formatting. Let's see what the training data looks like:

In [16]:
training.show()

+-------------------+--------------------+
|              label|            features|
+-------------------+--------------------+
| -9.490009878824548|(10,[0,1,2,3,4,5,...|
| 0.2577820163584905|(10,[0,1,2,3,4,5,...|
| -4.438869807456516|(10,[0,1,2,3,4,5,...|
|-19.782762789614537|(10,[0,1,2,3,4,5,...|
| -7.966593841555266|(10,[0,1,2,3,4,5,...|
| -7.896274316726144|(10,[0,1,2,3,4,5,...|
| -8.464803554195287|(10,[0,1,2,3,4,5,...|
| 2.1214592666251364|(10,[0,1,2,3,4,5,...|
| 1.0720117616524107|(10,[0,1,2,3,4,5,...|
|-13.772441561702871|(10,[0,1,2,3,4,5,...|
| -5.082010756207233|(10,[0,1,2,3,4,5,...|
|  7.887786536531237|(10,[0,1,2,3,4,5,...|
| 14.323146365332388|(10,[0,1,2,3,4,5,...|
|-20.057482615789212|(10,[0,1,2,3,4,5,...|
|-0.8995693247765151|(10,[0,1,2,3,4,5,...|
| -19.16829262296376|(10,[0,1,2,3,4,5,...|
|  5.601801561245534|(10,[0,1,2,3,4,5,...|
|-3.2256352187273354|(10,[0,1,2,3,4,5,...|
| 1.5299675726687754|(10,[0,1,2,3,4,5,...|
| -0.250102447941961|(10,[0,1,2,3,4,5,...|
+----------

This is the format that Spark expects. Two columns with the names "label" and "features". 

The "label" column then needs to have the numerical label, either a regression numerical value, or a numerical value that matches to a classification grouping. Later on we will talk about unsupervised learning algorithms that by their nature do not use or require a label.

The feature column has inside of it a vector of all the features that belong to that row. Usually what we end up doing is combining the various feature columns we have into a single 'features' column using the data transformations we've learned about.

Let's continue working through this simple example!

In [22]:
# These are the default values for the featuresCol, labelCol, predictionCol
lr = LinearRegression(featuresCol='features', labelCol='label', predictionCol='prediction')

# You could also pass in additional parameters for regularization

In [23]:
# Fit the model
lrModel = lr.fit(training)

In [24]:
# Print the coefficients and intercept for linear regression
print("Coefficients: {}".format(str(lrModel.coefficients))) # For each feature...
print('\n')
print("Intercept:{}".format(str(lrModel.intercept)))

Coefficients: [0.0073350710225801715,0.8313757584337543,-0.8095307954684084,2.441191686884721,0.5191713795290002,1.1534591903547016,-0.2989124112808717,-0.5128514186201779,-0.619712827067017,0.695615180432293]


Intercept:0.14228558260358093


There is a summary attribute that contains even more info!

In [25]:
# Summarize the model over the training set and print out some metrics
trainingSummary = lrModel.summary

Lots of info, here are a few examples:

In [26]:
trainingSummary.residuals.show()
print("RMSE: {}".format(trainingSummary.rootMeanSquaredError))
print("r2: {}".format(trainingSummary.r2))

+-------------------+
|          residuals|
+-------------------+
|-11.011130022096554|
| 0.9236590911176535|
|-4.5957401897776675|
|  -20.4201774575836|
|-10.339160314788181|
|-5.9552091439610555|
|-10.726906349283922|
| 2.1228071931912336|
|  4.077122222293811|
|-17.316168071241652|
| -4.593044343959059|
|  6.380476690746936|
| 11.320566035059846|
|-20.721971774534094|
| -2.736692773777401|
| -16.66886934252847|
|  8.242186378876315|
|-1.3723486332690233|
|-0.7060332131264666|
|-1.1591135969994064|
+-------------------+
only showing top 20 rows

RMSE: 10.16309157133015
r2: 0.027839179518600154


#### **RMSE and MSE**

Root Mean Square Error (RMSE) is a standard way to measure the error of a model in predicting quantitative data. RMSE can be thought of as some kind of (normalized) distance between the vector of predicted values and the vector of observed values.

The Mean Squared Error (MSE) measures how close a regression line is to a set of data points. It is a risk function corresponding to the expected value of the squared error loss. 
Mean square error is calculated by taking the average, specifically the mean, of errors squared from data as it relates to a function. 

## Train/Test Splits

But wait! We've commited a big mistake, we never separated our data set into a training and test set. Instead we trained on ALL of the data, something we generally want to avoid doing. Remember we won't get a fair evaluation of our model by judging how well it does again on the same data it was trained on!

Luckily Spark DataFrames have an almost too convienent method of splitting the data! Let's see it:

In [27]:
all_data = spark.read.format("libsvm").load("sample_linear_regression_data.txt")

In [28]:
# Pass in the split between training/test as a list.
# No correct, but generally 70/30 or 60/40 splits are used. 
# Depending on how much data you have and how unbalanced it is.
train_data,test_data = all_data.randomSplit([0.7,0.3])

In [ ]:
#display the training set
train_data.show()

In [ ]:
#display the test set
test_data.show()

In [33]:
unlabeled_data = test_data.select('features')

In [34]:
unlabeled_data.show()

+--------------------+
|            features|
+--------------------+
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
+--------------------+
only showing top 20 rows



Now we only train on the train_data

In [35]:
correct_model = lr.fit(train_data)

Now we can directly get a .summary object using the evaluate method:

In [36]:
test_results = correct_model.evaluate(test_data)

In [37]:
test_results.residuals.show()
print("RMSE: {}".format(test_results.rootMeanSquaredError))

+-------------------+
|          residuals|
+-------------------+
|-28.663931018754948|
| -25.06828485115981|
|-26.025230836199782|
|-19.273229013531733|
|   -20.854494693225|
| -18.53627115926748|
|-22.432858808830403|
|-19.298412717019172|
|-19.124520640077456|
| -16.92770283662761|
| -19.22039171801529|
|-18.126831048546872|
| -12.71053689576945|
|-12.624791726108159|
|-17.370423506587535|
|-12.786885331599386|
|-10.943635127382692|
|-10.816840593784196|
|-11.067523181564093|
| -8.601087720441043|
+-------------------+
only showing top 20 rows

RMSE: 10.392352996904318


Well that is nice, but realistically we will eventually want to test this model against unlabeled data, after all, that is the whole point of building the model in the first place. We can again do this with a convenient method call, in this case, transform(). Which was actually being called within the evaluate() method. Let's see it in action:

In [38]:
predictions = correct_model.transform(unlabeled_data)

In [39]:
predictions.show()

+--------------------+-------------------+
|            features|         prediction|
+--------------------+-------------------+
|(10,[0,1,2,3,4,5,...|0.09245214901151941|
|(10,[0,1,2,3,4,5,...|-2.9777331866168235|
|(10,[0,1,2,3,4,5,...|  3.075404900003708|
|(10,[0,1,2,3,4,5,...|-3.5642314033876077|
|(10,[0,1,2,3,4,5,...| 1.0717319036104616|
|(10,[0,1,2,3,4,5,...|-1.1310474561042425|
|(10,[0,1,2,3,4,5,...| 3.5869363359318207|
|(10,[0,1,2,3,4,5,...| 1.0231991510145415|
|(10,[0,1,2,3,4,5,...| 3.3438356074541566|
|(10,[0,1,2,3,4,5,...| 1.1956145643883649|
|(10,[0,1,2,3,4,5,...| 3.4968761049667196|
|(10,[0,1,2,3,4,5,...|  2.767286168714196|
|(10,[0,1,2,3,4,5,...|-2.6383342596098047|
|(10,[0,1,2,3,4,5,...|-0.7958030497825983|
|(10,[0,1,2,3,4,5,...|  4.448200403217114|
|(10,[0,1,2,3,4,5,...| 0.2283095427431973|
|(10,[0,1,2,3,4,5,...| -1.186718084905238|
|(10,[0,1,2,3,4,5,...|-1.2775106847510624|
|(10,[0,1,2,3,4,5,...|-0.7898271838653329|
|(10,[0,1,2,3,4,5,...|-2.4382600878127834|
+----------

Okay, so this data is a bit meaningless, so let's explore this same process with some data that actually makes a little more intuitive sense!

# Linear Regression (E-commerce example)

Basically what we do here is examine a dataset with Ecommerce Customer Data for a company's website and mobile app. Then we want to see if we can build a regression model that will predict the customer's yearly spend on the company's product.

First thing to do is start a Spark Session

In [40]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('lr_example').getOrCreate()

In [41]:
from pyspark.ml.regression import LinearRegression

In [42]:
# Use Spark to read in the Ecommerce Customers csv file.
data = spark.read.csv("Ecommerce_Customers.csv",inferSchema=True,header=True)

In [ ]:
# Print the Schema of the DataFrame
data.printSchema()

In [ ]:
data.show()

In [ ]:
data.head()

In [46]:
for item in data.head():
    print(item)

mstephenson@fernandez.com
835 Frank TunnelWrightmouth, MI 82180-9605
Violet
34.49726772511229
12.65565114916675
39.57766801952616
4.0826206329529615
587.9510539684005


## Setting Up DataFrame for Machine Learning 

In [47]:
# A few things we need to do before Spark can accept the data!
# It needs to be in the form of two columns
# ("label","features")

# Import VectorAssembler and Vectors
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [50]:
data.columns

['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

In [49]:
assembler = VectorAssembler(
    inputCols=["Avg Session Length", "Time on App", 
               "Time on Website",'Length of Membership'],
    outputCol="features")

In [52]:
output = assembler.transform(data)

In [55]:
output.select("features").show()

+--------------------+
|            features|
+--------------------+
|[34.4972677251122...|
|[31.9262720263601...|
|[33.0009147556426...|
|[34.3055566297555...|
|[33.3306725236463...|
|[33.8710378793419...|
|[32.0215955013870...|
|[32.7391429383803...|
|[33.9877728956856...|
|[31.9365486184489...|
|[33.9925727749537...|
|[33.8793608248049...|
|[29.5324289670579...|
|[33.1903340437226...|
|[32.3879758531538...|
|[30.7377203726281...|
|[32.1253868972878...|
|[32.3388993230671...|
|[32.1878120459321...|
|[32.6178560628234...|
+--------------------+
only showing top 20 rows



In [ ]:
output.show()

In [56]:
final_data = output.select("features",'Yearly Amount Spent')

In [57]:
train_data,test_data = final_data.randomSplit([0.7,0.3])

In [58]:
train_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                352|
|   mean| 500.38282291705366|
| stddev|  79.70951056372148|
|    min|   266.086340948469|
|    max|  765.5184619388373|
+-------+-------------------+



In [59]:
test_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                148|
|   mean| 496.77206393576085|
| stddev|  78.57802171061545|
|    min| 256.67058229005585|
|    max|  744.2218671047146|
+-------+-------------------+



In [60]:
# Create a Linear Regression Model object
lr = LinearRegression(labelCol='Yearly Amount Spent')

In [61]:
# Fit the model to the data and call this model lrModel
lrModel = lr.fit(train_data,)

In [62]:
# Print the coefficients and intercept for linear regression
print("Coefficients: {} Intercept: {}".format(lrModel.coefficients,lrModel.intercept))

Coefficients: [25.81957068884077,38.37787619186918,0.29501366416066854,61.84658452912895] Intercept: -1046.0312001285229


In [63]:
test_results = lrModel.evaluate(test_data)

In [64]:
# Interesting results....
test_results.residuals.show()

+--------------------+
|           residuals|
+--------------------+
| -3.9022847256887303|
| -3.3586251294190674|
|   4.640251769756219|
|  -6.243949689398107|
|-0.05748197528248511|
|  -3.904882962184274|
|  21.844298593632857|
|  3.2688014884624295|
| -3.9982704582219526|
|  -8.622267022892572|
|   4.691705294187216|
| -17.524558112566183|
|  18.162097529384255|
| -3.6624588175125723|
|   7.194250178582877|
|  -5.928668389799839|
|  -4.486577499463067|
|   7.093176848543806|
|  -2.311857287638759|
| -1.2917770528930532|
+--------------------+
only showing top 20 rows



In [65]:
unlabeled_data = test_data.select('features')

In [66]:
predictions = lrModel.transform(unlabeled_data)

In [67]:
predictions.show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[30.8364326747734...|471.40418515267834|
|[30.8794843441274...|493.56522511427374|
|[31.0472221394875...| 387.8571474192652|
|[31.0613251567161...| 493.7994077472997|
|[31.2606468698795...|421.38411323223386|
|[31.2681042107507...| 427.3754161360082|
|[31.2834474760581...| 569.9367908320346|
|[31.4459724827577...|481.60816344666614|
|[31.5171218025062...|279.91669110860767|
|[31.5261978982398...| 417.7167932152304|
|[31.5316044825729...|431.82390043517535|
|[31.5702008293202...| 563.4700502539711|
|[31.6005122003032...| 461.0107539617127|
|[31.6253601348306...|379.99935957443677|
|[31.6548096756927...|468.06917354896564|
|[31.7242025238451...|509.31655567776033|
|[31.7656188210424...| 501.0406591350702|
|[31.8209982016720...|417.58210416466954|
|[31.8530748017465...|461.59698074999073|
|[31.8627411090001...| 557.5899182269397|
+--------------------+------------

In [68]:
print("RMSE: {}".format(test_results.rootMeanSquaredError))
print("MSE: {}".format(test_results.meanSquaredError))

RMSE: 9.48720365069516
MSE: 90.00703310976354


Excellent results!